In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
import torch

def inference(network,test_loader):
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
        output = network(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
    accuracy = float(correct / len(test_loader.dataset))
    return accuracy

In [ ]:
import numpy as np


def FedAvg(models, weights=None):
    new_model = models[0]
    state_dicts = [model.state_dict() for model in models]
    state_dict = new_model.state_dict()
    for key in models[1].state_dict():
        state_dict[key] = torch.from_numpy(np.average([state[key].numpy() for state in state_dicts],
                                                      axis=0, 
                                                      weights=weights))
    new_model.load_state_dict(state_dict)
    return new_model

In [ ]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [ ]:
from torch import optim
from openfl.experimental.interface import FLSpec
from openfl.experimental.placement import aggregator, collaborator


class FederatedFlow(FLSpec):

    def __init__(self, model=None, optimizer=None, rounds=3, **kwargs):
        super().__init__(**kwargs)
        if model is not None:
            self.model = model
            self.optimizer = optimizer
        else:
            self.model = Net()
            self.optimizer = optim.SGD(self.model.parameters(), lr=learning_rate,
                                       momentum=momentum)
        self.rounds = rounds

    @aggregator
    def start(self):
        print(f'Performing initialization for model')
        self.collaborators = self.runtime.collaborators
        self.private = 10
        self.current_round = 0
        self.next(self.aggregated_model_validation, foreach='collaborators', exclude=['private'])

    @collaborator
    def aggregated_model_validation(self):
        print(f'Performing aggregated model validation for collaborator {self.input}')
        self.agg_validation_score = inference(self.model, self.test_loader)
        print(f'{self.input} value of {self.agg_validation_score}')
        self.next(self.train)

    @collaborator
    def train(self):
        self.model.train()
        self.optimizer = optim.SGD(self.model.parameters(), lr=learning_rate,
                                   momentum=momentum)
        train_losses = []
        for batch_idx, (data, target) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            self.optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: 1 [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    batch_idx * len(data), len(self.train_loader.dataset),
                    100. * batch_idx / len(self.train_loader), loss.item()))
                self.loss = loss.item()
                torch.save(self.model.state_dict(), 'model.pth')
                torch.save(self.optimizer.state_dict(), 'optimizer.pth')
        self.training_completed = True
        self.next(self.local_model_validation)

    @collaborator
    def local_model_validation(self):
        self.local_validation_score = inference(self.model, self.test_loader)
        print(
            f'Doing local model validation for collaborator {self.input}: {self.local_validation_score}')
        self.next(self.join, exclude=['training_completed'])

    @aggregator
    def join(self, inputs):
        self.average_loss = sum(input.loss for input in inputs) / len(inputs)
        self.aggregated_model_accuracy = sum(
            input.agg_validation_score for input in inputs) / len(inputs)
        self.local_model_accuracy = sum(
            input.local_validation_score for input in inputs) / len(inputs)
        print(f'Average aggregated model validation values = {self.aggregated_model_accuracy}')
        print(f'Average training loss = {self.average_loss}')
        print(f'Average local model validation values = {self.local_model_accuracy}')
        self.model = FedAvg([input.model for input in inputs])
        self.optimizer = [input.optimizer for input in inputs][0]
        self.current_round += 1
        if self.current_round < self.rounds:
            self.next(self.aggregated_model_validation,
                      foreach='collaborators', exclude=['private'])
        else:
            self.next(self.end)

    @aggregator
    def end(self):
        print(f'This is the end of the flow')

In [ ]:
model = None
best_model = None
optimizer = None
# FLSpec has access to runtime object, it will send following arguments to runtime to build plan.yaml file
flflow = FederatedFlow(model, optimizer, rounds=2, checkpoint=True)

In [ ]:
from openfl.experimental.interface import Federation
from openfl.experimental.runtime import FederatedRuntime
from openfl.experimental.interface import Aggregator
from openfl.experimental.interface import Collaborator

# 2. Connect to director and run service
# 3. Connect to envoys and run service
federation = Federation(
    director={
        "fqdn": "fqdn",
        "port": 1008,
        "user": "ubuntu",
        "password": "password for director machine",
        "identity file": "password OR identity file for this director",
    },
    envoys={
        "env1": {
            "fqdn": "envoy fqdn",
            "port": 1001,
            "username": "ubuntu",
            "password": "password for envoy 1",
            "identity_file": "password OR identity file for this envoy",
            "private_attributes": "env1_pa",
        },
    },
    tls=False,
    client_id="client_id",
    cert_chain="path to cert chain",
    api_cert="path to api cert",
    api_private_key="path to api private key",
)

agg = Aggregator()
collaborators = [Collaborator() for _ in range(2)]

# Job of Runtime is defined as below:
# 1. Populate plan.yaml, data.yaml
# 4. Provide metrics from aggregator -> director -> experiment mgr (here)
flflow.runtime = FederatedRuntime(
    aggregator=agg,
    collaborators=collaborators,
    federation=federation
)

In [ ]:
# 1. Prepare 2 zip files for 1 for aggregator, and 1 for collaborator
# aggregator zip will include:
    # flow.py (this file)
    # file mentioned in aggregator_private_attributes
    # plan.yaml
    # data.yaml
# collaborator zip will include:
    # file mentioned in private_attributes_filepath
    # plan.yaml
    # data.yaml
# 2. Submit experiment zip files to director
flflow.run()

In [ ]:
# FLSpec will do this via runtime object provided to it.
flflow.stream_metrics()

In [ ]:
# Get status of running experiment. Will use runtime object to get the latest status.
flflow.experiment_status()